In [12]:
# Dependencies
import psycopg2
import pandas as pd
from datetime import datetime

# Database credentials
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'adimalara',
    'user': 'postgres',
    'password': 'changeme-strong-pass'
}

# Connect to database
conn = psycopg2.connect(**DB_CONFIG)
print("Connected to database successfully!")

Connected to database successfully!


In [15]:
from datetime import datetime, timedelta

# ===== DISCOVER DATA RANGE =====
discovery_info_query = """
    SELECT 
        MIN(timestamp) as earliest,
        MAX(timestamp) as latest,
        MIN(latitude) as min_lat,
        MAX(latitude) as max_lat,
        MIN(longitude) as min_lon,
        MAX(longitude) as max_lon,
        COUNT(*) as total_records,
        COUNT(DISTINCT mmsi) as unique_vessels
    FROM ais1_position
"""

data_range = pd.read_sql_query(discovery_info_query, conn)

# Verify data exists
if data_range.empty or data_range['earliest'][0] is None:
    raise ValueError("No data in database!")

print("=== DATABASE OVERVIEW ===")
print(f"Total records: {data_range['total_records'][0]:,}")
print(f"Unique vessels: {data_range['unique_vessels'][0]:,}")
print(f"Time: {data_range['earliest'][0]} to {data_range['latest'][0]}")
print(f"Lat: [{data_range['min_lat'][0]:.2f}, {data_range['max_lat'][0]:.2f}]")
print(f"Lon: [{data_range['min_lon'][0]:.2f}, {data_range['max_lon'][0]:.2f}]")

# ===== CALCULATE TIME WINDOW =====
earliest = data_range['earliest'][0]
latest = data_range['latest'][0]
total_days = (latest - earliest).days
middle_date = earliest + timedelta(days=total_days // 2)

# Use 2 weeks for exploration (manageable data size)
START_TIME = middle_date - timedelta(days=7)
END_TIME = middle_date + timedelta(days=7)

# ===== USE FULL GEOGRAPHIC AREA =====
# Convert to Python float (fixes numpy type error)
LAT_MIN = float(data_range['min_lat'][0])
LAT_MAX = float(data_range['max_lat'][0])
LON_MIN = float(data_range['min_lon'][0])
LON_MAX = float(data_range['max_lon'][0])

NUM_VESSELS = 30

print("\n=== QUERY PARAMETERS ===")
print(f"Time: {START_TIME.date()} to {END_TIME.date()} ({(END_TIME - START_TIME).days} days)")
print(f"Area: Lat [{LAT_MIN:.3f}, {LAT_MAX:.3f}], Lon [{LON_MIN:.3f}, {LON_MAX:.3f}]")
print(f"Target vessels: {NUM_VESSELS}")

# ===== VERIFY DATA IN QUERY WINDOW =====
verify_query = """
    SELECT COUNT(*) as count
    FROM ais1_position
    WHERE timestamp BETWEEN %s AND %s
      AND latitude BETWEEN %s AND %s
      AND longitude BETWEEN %s AND %s
"""
verify_params = [START_TIME, END_TIME, LAT_MIN, LAT_MAX, LON_MIN, LON_MAX]
verify_result = pd.read_sql_query(verify_query, conn, params=verify_params)
print(f"\n✓ Records in query window: {verify_result['count'][0]:,}")

/tmp/ipykernel_23304/4184853877.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_range = pd.read_sql_query(discovery_info_query, conn)


=== DATABASE RANGE ===
Time: 2024-08-02 12:17:49.241000+00:00 to 2025-03-31 06:01:36.822000+00:00
Lat: [59.93, 61.19]
Lon: [5.01, 6.15]

=== QUERY PARAMETERS ===
Time: 2024-11-23 to 2024-12-07 (14 days)
Area: Lat [60.435, 60.685], Lon [5.466, 5.695]
Vessels: 30


In [16]:
# STEP 1: Find most active vessels in this area/time 
discovery_query = """
    SELECT 
        mmsi,
        COUNT(*) as record_count,
        MIN(timestamp) as first_seen,
        MAX(timestamp) as last_seen
    FROM ais1_position
    WHERE timestamp BETWEEN %s AND %s
      AND latitude BETWEEN %s AND %s
      AND longitude BETWEEN %s AND %s
    GROUP BY mmsi
    HAVING COUNT(*) > 50
    ORDER BY record_count DESC
    LIMIT %s
"""

discovery_params = [
    START_TIME, END_TIME,
    LAT_MIN, LAT_MAX,
    LON_MIN, LON_MAX,
    NUM_VESSELS
]

# Get vessel list
vessels_df = pd.read_sql_query(discovery_query, conn, params=discovery_params)

print(f"\n✓ Found {len(vessels_df)} vessels")
if len(vessels_df) > 0:
    print(f"Total records: {vessels_df['record_count'].sum():,}")
    print(f"Avg records per vessel: {vessels_df['record_count'].mean():.0f}")
    print("\nTop 5 most active:")
    print(vessels_df[['mmsi', 'record_count']].head())
    
    # Extract MMSI list
    MMSI_LIST = vessels_df['mmsi'].tolist()
else:
    print("⚠️  No vessels found. Try:")
    print("  - Lowering HAVING COUNT(*) threshold")
    print("  - Expanding time window")
    print("  - Checking data distribution")


Found 0 vessels
Total records: 0

Top 5 most active:
Empty DataFrame
Columns: [mmsi, record_count]
Index: []


/tmp/ipykernel_23304/782305164.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vessels_df = pd.read_sql_query(discovery_query, conn, params=discovery_params)
